# The wealth of Nations - Links Between Economic Health and Environmental Welfare

## Data analysis and collection
Exploration of available databases from World Bank API and selection of indicators

In [14]:
import wbgapi as wb
import pandas as pd
print(wb.source.info())

economies = list(wb.economy.list())
print(">>>>>>> NUMBER OF ECONOMIES IN THE DATABASE >>>>>>>")
print(len(economies))
print(economies[:5])

id    name                                                                  code      concepts  lastupdated
----  --------------------------------------------------------------------  ------  ----------  -------------
1     Doing Business                                                        DBS              3  2021-08-18
2     World Development Indicators                                          WDI              3  2025-10-07
3     Worldwide Governance Indicators                                       WGI              3  2024-11-05
5     Subnational Malnutrition Database                                     SNM              3  2016-03-21
6     International Debt Statistics                                         IDS              4  2025-02-26
11    Africa Development Indicators                                         ADI              3  2013-02-22
12    Education Statistics                                                  EDS              3  2024-06-25
13    Enterprise Surveys         

In [ ]:
economies = list(wb.economy.list())
print(len(economies))
print(economies[:5])

266
[{'id': 'ABW', 'value': 'Aruba', 'aggregate': False, 'longitude': -70.0167, 'latitude': 12.5167, 'region': 'LCN', 'adminregion': '', 'lendingType': 'LNX', 'incomeLevel': 'HIC', 'capitalCity': 'Oranjestad'}, {'id': 'AFE', 'value': 'Africa Eastern and Southern', 'aggregate': True, 'longitude': None, 'latitude': None, 'region': '', 'adminregion': '', 'lendingType': '', 'incomeLevel': '', 'capitalCity': ''}, {'id': 'AFG', 'value': 'Afghanistan', 'aggregate': False, 'longitude': 69.1761, 'latitude': 34.5228, 'region': 'MEA', 'adminregion': 'MNA', 'lendingType': 'IDX', 'incomeLevel': 'LIC', 'capitalCity': 'Kabul'}, {'id': 'AFW', 'value': 'Africa Western and Central', 'aggregate': True, 'longitude': None, 'latitude': None, 'region': '', 'adminregion': '', 'lendingType': '', 'incomeLevel': '', 'capitalCity': ''}, {'id': 'AGO', 'value': 'Angola', 'aggregate': False, 'longitude': 13.242, 'latitude': -8.81155, 'region': 'SSF', 'adminregion': 'SSA', 'lendingType': 'IBD', 'incomeLevel': 'LMC', 

Finding the indicators related to the economy such as GDP or GDP growth.

In [18]:
WDI = 2     # second database = WDI (World Development Indicators)

for ind in wb.series.list(db = WDI, q = ('gdp')):
    print(f"{ind['id']}: {ind['value']}")


EG.GDP.PUSE.KO.PP: GDP per unit of energy use (PPP $ per kg of oil equivalent)
EG.GDP.PUSE.KO.PP.KD: GDP per unit of energy use (constant 2021 PPP $ per kg of oil equivalent)
EG.USE.COMM.GD.PP.KD: Energy use (kg of oil equivalent) per $1,000 GDP (constant 2021 PPP)
EN.GHG.CO2.RT.GDP.KD: Carbon intensity of GDP (kg CO2e per constant 2015 US$ of GDP)
EN.GHG.CO2.RT.GDP.PP.KD: Carbon intensity of GDP (kg CO2e per 2021 PPP $ of GDP)
NY.GDP.DEFL.KD.ZG: Inflation, GDP deflator (annual %)
NY.GDP.DEFL.KD.ZG.AD: Inflation, GDP deflator: linked series (annual %)
NY.GDP.DEFL.ZS: GDP deflator (base year varies by country)
NY.GDP.DEFL.ZS.AD: GDP deflator: linked series (base year varies by country)
NY.GDP.DISC.CN: Discrepancy in expenditure estimate of GDP (current LCU)
NY.GDP.DISC.KN: Discrepancy in expenditure estimate of GDP (constant LCU)
NY.GDP.MKTP.CD: GDP (current US$)
NY.GDP.MKTP.CN: GDP (current LCU)
NY.GDP.MKTP.CN.AD: GDP: linked series (current LCU)
NY.GDP.MKTP.KD: GDP (constant 2015 US$)

Looking over health indicators such as life expectancy, mortality and health expenditure which links economy to health of the population.

In [ ]:
for query in ['life expectancy', 'mortality', 'health expenditure']:
    for ind in wb.series.list(db = WDI, q = query):
        print(f"{ind['id']}: {ind['value']}")


SP.DYN.LE00.FE.IN: Life expectancy at birth, female (years)
SP.DYN.LE00.IN: Life expectancy at birth, total (years)
SP.DYN.LE00.MA.IN: Life expectancy at birth, male (years)
SH.DYN.MORT: Mortality rate, under-5 (per 1,000 live births)
SH.DYN.MORT.FE: Mortality rate, under-5, female (per 1,000 live births)
SH.DYN.MORT.MA: Mortality rate, under-5, male (per 1,000 live births)
SH.DYN.NCOM.FE.ZS: Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70, female (%)
SH.DYN.NCOM.MA.ZS: Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70, male (%)
SH.DYN.NCOM.ZS: Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%)
SH.DYN.NMRT: Mortality rate, neonatal (per 1,000 live births)
SH.STA.AIRP.FE.P5: Mortality rate attributed to household and ambient air pollution, age-standardized, female (per 100,000 female population)
SH.STA.AIRP.MA.P5: Mortality rate attributed to household and ambient air pollution, age-standardized, male (per 100,000

Examining possible indicators for environmental welfare for a country focusing on pollution, use of renewable energy or access to clean water.

In [ ]:

for query in ['CO2', 'pollution', 'GHG', 'renewable energy', 'forest', 'water']:
    for ind in wb.series.list(db = WDI, q = query):
        print(f"{ind['id']}: {ind['value']}")


EN.GHG.CO2.AG.MT.CE.AR5: Carbon dioxide (CO2) emissions from Agriculture (Mt CO2e)
EN.GHG.CO2.BU.MT.CE.AR5: Carbon dioxide (CO2) emissions from Building (Energy) (Mt CO2e)
EN.GHG.CO2.FE.MT.CE.AR5: Carbon dioxide (CO2) emissions from Fugitive Emissions (Energy) (Mt CO2e)
EN.GHG.CO2.IC.MT.CE.AR5: Carbon dioxide (CO2) emissions from Industrial Combustion (Energy) (Mt CO2e)
EN.GHG.CO2.IP.MT.CE.AR5: Carbon dioxide (CO2) emissions from Industrial Processes (Mt CO2e)
EN.GHG.CO2.LU.DF.MT.CE.AR5: Carbon dioxide (CO2) net fluxes from LULUCF - Deforestation (Mt CO2e)
EN.GHG.CO2.LU.FL.MT.CE.AR5: Carbon dioxide (CO2) net fluxes from LULUCF - Forest Land (Mt CO2e)
EN.GHG.CO2.LU.MT.CE.AR5: Carbon dioxide (CO2) net fluxes from LULUCF - Total excluding non-tropical fires (Mt CO2e)
EN.GHG.CO2.LU.OL.MT.CE.AR5: Carbon dioxide (CO2) net fluxes from LULUCF - Other Land (Mt CO2e)
EN.GHG.CO2.LU.OS.MT.CE.AR5: Carbon dioxide (CO2) net fluxes from LULUCF - Organic Soil (Mt CO2e)
EN.GHG.CO2.MT.CE.AR5: Carbon diox

## Choosing main indicators for each group
### Economy 
GDP per capita